# Comment/Annotation Nanopublication Generator (Corrected)

Creates paper annotation nanopublications with quotations and interpretations.

**Template:** [Paper Annotation Template](https://w3id.org/np/RA24onqmqTMsraJ7ypYFOuckmNWpo4Zv5gsLqhXt7xYPU)

In [ ]:
CONFIG_FILE = "../config/vbae208/vbae208_comment.json"
OUTPUT_DIR = "../output/comment"

In [ ]:
import json
from pathlib import Path
from datetime import datetime, timezone
from rdflib import Dataset, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, FOAF

NP = Namespace("http://www.nanopub.org/nschema#")
DCT = Namespace("http://purl.org/dc/terms/")
NT = Namespace("https://w3id.org/np/o/ntemplate/")
NPX = Namespace("http://purl.org/nanopub/x/")
PROV = Namespace("http://www.w3.org/ns/prov#")
ORCID = Namespace("https://orcid.org/")
CITO = Namespace("http://purl.org/spar/cito/")

COMMENT_TEMPLATE = URIRef("https://w3id.org/np/RA24onqmqTMsraJ7ypYFOuckmNWpo4Zv5gsLqhXt7xYPU")
PROV_TEMPLATE = URIRef("https://w3id.org/np/RA7lSq6MuK_TIC6JMSHvLtee3lpLoZDOqLJCLXevnrPoU")
PUBINFO_TEMPLATE_1 = URIRef("https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw")
PUBINFO_TEMPLATE_2 = URIRef("https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI")

print("✓ Setup complete")

In [ ]:
with open(CONFIG_FILE, 'r', encoding='utf-8') as f:
    config = json.load(f)

metadata = config.get('metadata', {})
print(f"✓ Loaded {len(config['nanopublications'])} comment nanopubs")

In [ ]:
def create_comment_nanopub(np_config, metadata):
    TEMP_NP = Namespace("http://purl.org/nanopub/temp/np/")
    
    this_np = URIRef("http://purl.org/nanopub/temp/np/")
    head_graph = URIRef("http://purl.org/nanopub/temp/np/Head")
    assertion_graph = URIRef("http://purl.org/nanopub/temp/np/assertion")
    provenance_graph = URIRef("http://purl.org/nanopub/temp/np/provenance")
    pubinfo_graph = URIRef("http://purl.org/nanopub/temp/np/pubinfo")
    
    author_uri = ORCID[metadata['creator_orcid']]
    
    # Paper URI
    paper_doi = np_config['paper_doi']
    paper_uri = URIRef(paper_doi if paper_doi.startswith('http') else f"https://doi.org/{paper_doi}")
    
    ds = Dataset()
    ds.bind("this", "http://purl.org/nanopub/temp/np/")
    ds.bind("sub", TEMP_NP)
    ds.bind("np", NP)
    ds.bind("dct", DCT)
    ds.bind("nt", NT)
    ds.bind("npx", NPX)
    ds.bind("xsd", XSD)
    ds.bind("rdfs", RDFS)
    ds.bind("orcid", ORCID)
    ds.bind("prov", PROV)
    ds.bind("foaf", FOAF)
    ds.bind("cito", CITO)
    
    # HEAD
    head = ds.graph(head_graph)
    head.add((this_np, RDF.type, NP.Nanopublication))
    head.add((this_np, NP.hasAssertion, assertion_graph))
    head.add((this_np, NP.hasProvenance, provenance_graph))
    head.add((this_np, NP.hasPublicationInfo, pubinfo_graph))
    
    # ASSERTION
    assertion = ds.graph(assertion_graph)
    
    # Paper with quoted text and comment
    assertion.add((paper_uri, CITO.hasQuotedText, Literal(np_config['quotation'])))
    assertion.add((paper_uri, RDFS.comment, Literal(np_config['comment'])))
    
    # isPartOf (paper is part of systematic review)
    is_part_of = metadata.get('is_part_of', {})
    if is_part_of.get('uri'):
        assertion.add((paper_uri, DCT.isPartOf, URIRef(is_part_of['uri'])))
    
    # ORCID quotes the paper
    assertion.add((author_uri, CITO.quotes, paper_uri))
    
    # PROVENANCE
    provenance = ds.graph(provenance_graph)
    provenance.add((assertion_graph, PROV.wasAttributedTo, author_uri))
    
    # PUBINFO
    pubinfo = ds.graph(pubinfo_graph)
    pubinfo.add((author_uri, FOAF.name, Literal(metadata['creator_name'])))
    
    now = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.000Z")
    pubinfo.add((this_np, DCT.created, Literal(now, datatype=XSD.dateTime)))
    pubinfo.add((this_np, DCT.creator, author_uri))
    pubinfo.add((this_np, DCT.license, URIRef("https://creativecommons.org/licenses/by/4.0/")))
    pubinfo.add((this_np, NPX.wasCreatedAt, URIRef("https://sciencelive4all.org/")))
    pubinfo.add((this_np, NPX.hasNanopubType, CITO.cites))
    
    # Label (truncated quotation)
    quotation = np_config['quotation']
    label_text = quotation[:50] + '...' if len(quotation) > 50 else quotation
    label = f"Paper annotation: {label_text}"
    pubinfo.add((this_np, RDFS.label, Literal(label)))
    
    pubinfo.add((this_np, NT.wasCreatedFromTemplate, COMMENT_TEMPLATE))
    pubinfo.add((this_np, NT.wasCreatedFromProvenanceTemplate, PROV_TEMPLATE))
    pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_1))
    pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_2))
    
    return ds, label

print("✓ Function defined")

In [ ]:
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
generated_files = []

for np_config in config['nanopublications']:
    ds, label = create_comment_nanopub(np_config, metadata)
    output_file = Path(OUTPUT_DIR) / f"{np_config['id']}.trig"
    ds.serialize(destination=str(output_file), format='trig')
    generated_files.append(output_file)
    print(f"✓ Generated: {output_file}")

print(f"\nTotal: {len(generated_files)} nanopublications")

In [ ]:
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())